### DATA 302 - Final Project
#### Due Dec 17 by 11:59pm

Submit a .data302 file as per usual.

Make sure you've completed all of the validation exercises first!

Carefully double check all of you work before submitting. Remember the common errors we've discussed throughout the semester.

Don't just trust your first answer. Write additional queries to help you spot-check your results.

One of these questions is extra credit! I.e., the project is worth 80 points, but if you answer all questions correctly you will earn 90 points. You're welcome!

In [1]:
# Library imports
import sqlite3
import pandas as pd

# Additional imports needed to create and validate your .data302 files
import json
from collections import defaultdict as dd

# This file is posted on Blackboard
from dbga import validate_submission_format

In [2]:
# Create dictionary to store submission
# Create a default dictionary whose default value will be an empty dictionary
my_sql = dd(dict)

# If you would like to create any views, each CREATE VIEW statement
# will be stored in a list named 'setup'
setup = []

# Add the setup list to the dictionary. The key MUST BE 'setup':
my_sql['setup'] = setup

In [3]:
# connect to database
conn = sqlite3.connect('data/hardware.sqlite')

curs = conn.cursor()

curs.execute("PRAGMA foreign_keys = ON;")

In [4]:
# look at different tables
sql = """
SELECT * FROM sqlite_master
;"""

pd.read_sql(sql, conn)

,type,name,tbl_name,rootpage,sql
0,table,tState,tState,2,CREATE TABLE tState (\n state_i...
1,index,sqlite_autoindex_tState_1,tState,3,None
2,table,tZip,tZip,4,CREATE TABLE tZip (\n zip TEXT ...
3,index,sqlite_autoindex_tZip_1,tZip,5,None
4,table,tProd,tProd,6,CREATE TABLE tProd (\n prod_id ...
5,table,tCust,tCust,7,CREATE TABLE tCust (\n cust_id ...
6,table,sqlite_sequence,sqlite_sequence,8,"CREATE TABLE sqlite_sequence(name,seq)"
7,table,tInvoice,tInvoice,9,CREATE TABLE tInvoice (\n invoi...
8,table,tInvoiceDetail,tInvoiceDetail,10,CREATE TABLE tInvoiceDetail (\n ...
9,index,sqlite_autoindex_tInvoiceDetail_1,tInvoiceDetail,11,None


---
1) Generate a summary, by month and year of how our store is performing.

Have your query return the following:
 - year
 - month
 - Sales: total sales for the month (i.e., sum of qty * unit price). Don't forget to divide by 100 so the results are in dollars rather than pennies.
 - NumInvoices: number of invoice_id (not invoice lines) for the month
 - NumCust: number of _distinct_ customers who made a purchase (i.e. only count the customer at most once per month)
 - InvoicesPerCust: average number of invoices per customer (i.e. NumInvoices/NumCust)
 - SalesPerCust: average sales per customer (i.e. Sales/NumCust)
 - SalesPerInvoice: average sales per order (i.e. Sales/NumInvoices)

Order the results by year ASC, month ASC

_Hint: Watch out for integer division!_

In [126]:
# Testing CTE
sql1a = """
SELECT year,
        month,
        qty,
        unit_price,
        invoice_id,
        cust_id
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
;"""

pd.read_sql(sql1a, conn)

,year,month,qty,unit_price,invoice_id,cust_id
0,2016,1,1,100,1,1
1,2016,1,1,499,1,1
2,2016,1,1,399,1,1
3,2016,1,1,3599,1,1
4,2016,1,1,1800,1,1
...,...,...,...,...,...,...
105313,2024,11,1,199,28550,28
105314,2024,11,1,1698,28550,28
105315,2024,11,1,1099,28550,28
105316,2024,11,1,1200,28550,28


In [128]:
sql1 = """
WITH InvoiceData AS
(SELECT year,
        month,
        qty,
        unit_price,
        invoice_id,
        cust_id
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
)

SELECT year,
        month,
        SUM(qty * unit_price)/100 AS Sales,
        COUNT(DISTINCT invoice_id) AS NumInvoices,
        COUNT(DISTINCT cust_id) AS NumCust,
        COUNT(DISTINCT invoice_id)*1.0/COUNT(DISTINCT cust_id) AS InvoicesPerCust,
        (SUM(qty * unit_price)/100)*1.0/COUNT(DISTINCT cust_id) AS SalesPerCust,
        (SUM(qty * unit_price)/100)*1.0/COUNT(DISTINCT invoice_id) AS SalesPerInvoice
FROM InvoiceData
GROUP BY year, month
ORDER BY year ASC, month ASC
;"""

pd.read_sql(sql1, conn)

,year,month,Sales,NumInvoices,NumCust,InvoicesPerCust,SalesPerCust,SalesPerInvoice
0,2016,1,10279,148,124,1.193548,82.895161,69.452703
1,2016,2,12277,149,126,1.182540,97.436508,82.395973
2,2016,3,14079,153,131,1.167939,107.473282,92.019608
3,2016,4,12915,181,131,1.381679,98.587786,71.353591
4,2016,5,12788,157,130,1.207692,98.369231,81.452229
...,...,...,...,...,...,...,...,...
102,2024,7,28892,364,214,1.700935,135.009346,79.373626
103,2024,8,22465,271,214,1.266355,104.976636,82.896679
104,2024,9,28194,361,212,1.702830,132.990566,78.099723
105,2024,10,35045,370,213,1.737089,164.530516,94.716216


In [129]:
# store question 1
my_sql['q1']['sql'] = sql1

---
2) For each year in the data, determine the month that had the lowest sales.

Since there are 9 years of data, this should return 9 records.

Return:
- year
- month
- sales (sum of qty * unit_price). Don't forget to divide by 100

Order by year ASC, month ASC

In [133]:
# CTE testing
sql2a = """
WITH InvoiceData AS
(SELECT year,
        month,
        (SUM(qty * unit_price)/100) AS sales
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY year, month
)

SELECT year,
        month,
        Sales,
        Min(Sales) as MinSales
FROM InvoiceData
GROUP BY year, month
;"""
pd.read_sql(sql2a, conn)

,year,month,sales,MinSales
0,2016,1,10279,10279
1,2016,2,12277,12277
2,2016,3,14079,14079
3,2016,4,12915,12915
4,2016,5,12788,12788
...,...,...,...,...
102,2024,7,28892,28892
103,2024,8,22465,22465
104,2024,9,28194,28194
105,2024,10,35045,35045


In [136]:
sql2 = """
WITH InvoiceData AS
(SELECT year,
        month,
        (SUM(qty * unit_price)/100) AS sales
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY year, month
),

MinSalesPerYear AS
(SELECT year,
        month,
        sales,
        Min(Sales) as MinSales
FROM InvoiceData
GROUP BY year
)

SELECT year,
        month,
        sales
FROM MinSalesPerYear
ORDER BY year ASC, month ASC
;"""
pd.read_sql(sql2, conn)

,year,month,sales
0,2016,1,10279
1,2017,1,11721
2,2018,1,12369
3,2019,10,16027
4,2020,6,19188
5,2021,9,21865
6,2022,1,19068
7,2023,5,25203
8,2024,8,22465


In [15]:
# confirm results
sql2b = """
SELECT year,
        month,
        (SUM(qty * unit_price)/100) AS Sales
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY year, month
;"""
pd.set_option('display.max_rows', None)  # Set to None to show all rows
pd.read_sql(sql2b, conn)

,year,month,Sales
0,2016,1,10279
1,2016,2,12277
2,2016,3,14079
3,2016,4,12915
4,2016,5,12788
5,2016,6,13232
6,2016,7,12510
7,2016,8,12638
8,2016,9,12749
9,2016,10,12209


In [137]:
# store question 2
my_sql['q2']['sql'] = sql2

In [17]:
# reset max row display
pd.reset_option('display.max_rows')

---

3. Are there any products we haven't sold at least one of in each month? If so, report the product as well as each year/month when it was not sold.

Return:

- prod_id
- prod_desc
- unit_price (divide it by 100)
- year
- month

Order the results by year ASC, month ASC, unit_price DESC

In [18]:
sql3a = """
SELECT * FROM tProd
;"""
pd.read_sql(sql3a, conn)

,prod_id,prod_desc,unit_price
0,501,Washer,10
1,502,Nail,25
2,503,Nut,25
3,504,Screw,50
4,505,Bolt,100
5,506,Sandpaper,300
6,507,Paint,1800
7,508,Bradawl,199
8,509,Clamp,299
9,510,Screwdriver,399


In [19]:
sql3b = """
SELECT prod_id,
        prod_desc,
        qty,
        unit_price*1.0/100 as unit_price,
        year,
        month
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY prod_id, year, month
ORDER BY year ASC, month ASC, unit_price DESC
;"""
pd.read_sql(sql3b, conn)

,prod_id,prod_desc,qty,unit_price,year,month
0,530,Chainsaw,1,499.99,2016,1
1,529,Workbench,1,300.00,2016,1
2,528,Ladder,1,80.00,2016,1
3,521,Drill,1,69.00,2016,1
4,527,Toolbox,1,50.00,2016,1
...,...,...,...,...,...,...
3203,505,Bolt,1,1.00,2024,11
3204,504,Screw,1,0.50,2024,11
3205,502,Nail,1,0.25,2024,11
3206,503,Nut,1,0.25,2024,11


In [138]:
sql3 = """
WITH YearMonthCombo AS
(SELECT DISTINCT year, month
FROM tInvoice
),

ProductsWithSales AS
(SELECT prod_id,
        year,
        month
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
GROUP BY prod_id, year, month
ORDER BY year ASC, month ASC
)

SELECT 
    p.prod_id,
    p.prod_desc,
    p.unit_price * 1.0 / 100 as unit_price,
    ym.year,
    ym.month
FROM YearMonthCombo ym
CROSS JOIN tProd p
LEFT JOIN ProductsWithSales pws ON p.prod_id = pws.prod_id 
    AND ym.year = pws.year 
    AND ym.month = pws.month
WHERE pws.prod_id IS NULL
ORDER BY ym.year ASC, ym.month ASC, unit_price DESC
;"""
pd.read_sql(sql3, conn)

,prod_id,prod_desc,unit_price,year,month
0,530,Chainsaw,499.99,2016,6
1,530,Chainsaw,499.99,2017,1


In [139]:
my_sql['q3']['sql'] = sql3

---

4. What are our top 5 selling products (in terms of total dollars sold)?

Return:
- prod_id
- prod_desc
- total qty sold
- total sales (don't forget to divide by 100)

Order by the total sales DESC

In [22]:
# testing
sql4a = """
SELECT prod_id,
        prod_desc,
        unit_price,
        SUM(qty) AS "total qty sold",
        SUM(qty * unit_price)*1.0/100 AS "total sales"
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY prod_id
;"""
pd.read_sql(sql4a, conn)

,prod_id,prod_desc,unit_price,total qty sold,total sales
0,501,Washer,10,4966,496.60
1,502,Nail,25,4770,1192.50
2,503,Nut,25,4845,1211.25
3,504,Screw,50,4977,2488.50
4,505,Bolt,100,4744,4744.00
5,506,Sandpaper,300,4513,13539.00
6,507,Paint,1800,3884,69912.00
7,508,Bradawl,199,4657,9267.43
8,509,Clamp,299,4585,13709.15
9,510,Screwdriver,399,4474,17851.26


In [140]:
sql4 = """
SELECT prod_id,
        prod_desc,
        SUM(qty) AS "total qty sold",
        SUM(qty * unit_price)*1.0/100 AS "total sales"
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY prod_id
ORDER BY "total sales" DESC
LIMIT 5
;"""
pd.read_sql(sql4, conn)

,prod_id,prod_desc,total qty sold,total sales
0,529,Workbench,910,273000.00
1,528,Ladder,2912,232960.00
2,530,Chainsaw,447,223495.53
3,521,Drill,3164,218316.00
4,527,Toolbox,3342,167100.00


In [141]:
my_sql['q4']['sql'] = sql4

---

5. The marketing department called. They want a mailing list of all customers who didn't purchase anything last month.

Make sure you don't hard-code the month into the query. It is likely that they will want this list every month moving forward on a fully automated basis.

Return:

- first
- last
- address
- city
- state_id
- zip

Order by first ASC, last ASC

In [25]:
# Testing CTE
sql5a = """
SELECT DISTINCT(cust_id)
FROM tCust
;"""
pd.read_sql(sql5a, conn)

,cust_id
0,1
1,2
2,3
3,4
4,5
...,...
246,247
247,248
248,249
249,250


In [26]:
# Testing
sql5b = """
SELECT *
FROM tInvoice
;"""
pd.read_sql(sql5b, conn)

,invoice_id,cust_id,year,month,day,time
0,1,1,2016,1,1,09:18:00
1,2,2,2016,1,1,09:58:00
2,3,3,2016,1,1,11:53:00
3,4,4,2016,1,1,12:21:00
4,5,5,2016,1,1,13:51:00
...,...,...,...,...,...,...
28545,28546,124,2024,11,30,13:00:00
28546,28547,242,2024,11,30,13:16:00
28547,28548,28,2024,11,30,15:18:00
28548,28549,199,2024,11,30,16:36:00


In [27]:
sql5c = ("""
SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
;""")
pd.read_sql(sql5c, conn)

,year,month
0,2024,11


In [142]:
sql5 = """
WITH LastMonth AS
(SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
),

LastMonthCustomers AS
(SELECT DISTINCT(cust_id)
FROM tInvoice
WHERE (year, month) IN (SELECT year, month FROM LastMonth)
),

NonLastMonthCustomers AS
(SELECT cust_id
FROM tCust
WHERE cust_id NOT IN (SELECT cust_id FROM LastMonthCustomers)
)

SELECT first,
        last,
        address,
        city,
        state_id,
        zip
FROM tCust
LEFT JOIN tZip USING (zip)
WHERE cust_id IN (SELECT cust_id FROM NonLastMonthCustomers)
ORDER BY first ASC, last ASC
;"""
pd.read_sql(sql5, conn)

,first,last,address,city,state_id,zip
0,Bala-Tik,Elliott,9088 Valley Road,Bayport,MN,55003
1,Bala-Tik,Ingram,1819 Pheasant Run,Moca,PR,676
2,Bib Fortuna,Zhang,9177 West Avenue,Belen,NM,87002
3,C-3Po,Rivera,6661 Strawberry Lane,Granger,WY,82934
4,Captain Antilles,Curtis,2327 Prospect Street,Miami,FL,33127
5,Captain Antilles,Walters,8754 2nd Street,Douglassville,TX,75560
6,Clone Commander Cody,Francis,6510 Sycamore Street,West Stewartstown,NH,3597
7,Clone Commander Cody,Greene,1220 Route 44,Milmine,IL,61855
8,Colonel Datoo,Jennings,2331 Jackson Avenue,Brooklyn,MD,21225
9,Darth Maul,Curtis,8779 Front Street,Weston,NE,68070


In [29]:
# checking
sql5d = """
SELECT COUNT(DISTINCT cust_id) 
FROM tCust
;"""
pd.read_sql(sql5d, conn)

,COUNT(DISTINCT cust_id)
0,251


In [30]:
# checking
sql5e = """
WITH LastMonth AS
(SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
),

LastMonthCustomers AS
(SELECT DISTINCT(cust_id)
FROM tInvoice
WHERE (year, month) IN (SELECT year, month FROM LastMonth)
)

SELECT COUNT(DISTINCT cust_id) FROM LastMonthCustomers  -- Purchased last month
;"""
pd.read_sql(sql5e, conn)

,COUNT(DISTINCT cust_id)
0,213


In [31]:
# checking
sql5f = """
WITH LastMonth AS
(SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
),

LastMonthCustomers AS
(SELECT DISTINCT(cust_id)
FROM tInvoice
WHERE (year, month) IN (SELECT year, month FROM LastMonth)
),

NonLastMonthCustomers AS
(SELECT cust_id
FROM tCust
WHERE cust_id NOT IN (SELECT cust_id FROM LastMonthCustomers)
)

SELECT COUNT(cust_id) FROM NonLastMonthCustomers  -- Didn't purchase last month
;"""
pd.read_sql(sql5f, conn)

,COUNT(cust_id)
0,38


In [32]:
# double checking names
sql5g = """
WITH LastMonth AS (
    SELECT DISTINCT year, month
    FROM tInvoice
    ORDER BY year DESC, month DESC
    LIMIT 1
)
SELECT DISTINCT cust_id,
                first,
                last
FROM tInvoice
JOIN tCust USING (cust_id)
WHERE (year, month) IN (SELECT year, month FROM LastMonth)
    AND first = "Lama Su"
;"""
pd.read_sql(sql5g, conn)

,cust_id,first,last
0,31,Lama Su,Vincent
1,64,Lama Su,Vincent
2,230,Lama Su,Doyle
3,36,Lama Su,Jennings
4,246,Lama Su,Schmidt


In [33]:
# logic checks out
38 + 213

251

In [143]:
my_sql['q5']['sql'] = sql5

---

6. Who is our top customer (in terms of total dollars sold)? Get a report of all the products they've purchased from us.

Do not hardcode the customer into the query. Again, we'll want this to be reusable in the future without having to adjust any query parameters.

Return:

- first
- last
- address
- zip
- prod_id
- prod_desc
- total qty
- total sales (don't forget to divide by 100)

Order by total sales DESC

In [35]:
# testing
sql6a = """
SELECT cust_id,
        (SUM(qty * unit_price)*1.0/100) AS "total sales"
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY cust_id
ORDER BY "total sales" DESC
LIMIT 1
;"""
pd.read_sql(sql6a, conn)

,cust_id,total sales
0,24,54031.53


In [144]:
sql6 = """
WITH TopCustomer AS
(SELECT cust_id,
        (SUM(qty * unit_price)*1.0/100) AS "total sales"
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY cust_id
ORDER BY "total sales" DESC
LIMIT 1
)

SELECT first,
        last,
        address,
        zip,
        prod_id,
        prod_desc,
        SUM(qty) AS "total qty",
        (SUM(qty * unit_price)*1.0/100) AS "total sales"
FROM tProd
JOIN tInvoiceDetail USING (prod_id)
JOIN tInvoice USING (invoice_id)
JOIN tCust USING (cust_id)
WHERE cust_id = (SELECT cust_id FROM TopCustomer)
GROUP BY prod_id
ORDER BY "total sales" DESC
;"""
pd.read_sql(sql6, conn)

,first,last,address,zip,prod_id,prod_desc,total qty,total sales
0,Snap,Mcbride,5524 Hawthorne Lane,52746,530,Chainsaw,37,18499.63
1,Snap,Mcbride,5524 Hawthorne Lane,52746,529,Workbench,46,13800.00
2,Snap,Mcbride,5524 Hawthorne Lane,52746,528,Ladder,38,3040.00
3,Snap,Mcbride,5524 Hawthorne Lane,52746,521,Drill,42,2898.00
4,Snap,Mcbride,5524 Hawthorne Lane,52746,527,Toolbox,41,2050.00
5,Snap,Mcbride,5524 Hawthorne Lane,52746,525,Hatchet,44,1583.56
6,Snap,Mcbride,5524 Hawthorne Lane,52746,526,Wire,40,1480.00
7,Snap,Mcbride,5524 Hawthorne Lane,52746,519,Monkeywrench,46,1377.70
8,Snap,Mcbride,5524 Hawthorne Lane,52746,524,Axe,46,1287.54
9,Snap,Mcbride,5524 Hawthorne Lane,52746,523,Saw,49,1028.51


In [145]:
my_sql['q6']['sql'] = sql6

---

7) Which customers have never purchased one or more of our products? Generate a list of all customer/product combinations that do not occur in our sales history.

Return:

- first
- last
- addr
- zip
- prod_id
- prod_desc

Order by first, last, addr, zip, prod_id - all ascending

In [40]:
# all customers
sql7a = """
SELECT first,
        last,
        address as addr,
        zip, 
        cust_id
FROM tCust
;"""
pd.read_sql(sql7a, conn)

,first,last,addr,zip,cust_id
0,Fode/Beed,Vincent,8509 Route 44,97846,1
1,Snap,Garza,9534 9th Street,4983,2
2,Unkar Plutt,Benson,3640 Magnolia Court,40921,3
3,Lama Su,Garza,179 Franklin Court,21675,4
4,Fode/Beed,Francis,9993 Inverness Drive,27820,5
...,...,...,...,...,...
246,Captain Antilles,Adams,8402 Lake Street,4574,247
247,C-3Po,Pollard,3934 Main Street,66503,248
248,Senator Ask Aak,Ingram,3141 Sycamore Drive,70789,249
249,Padme,Le,1031 Pheasant Run,89178,250


In [77]:
# every customers product purchases
sql7b = """
SELECT first,
        last,
        address as addr,
        zip, 
        cust_id,
        prod_id,
        prod_desc
FROM tCust
JOIN tInvoice USING (cust_id)
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY cust_id, prod_id
;"""
pd.read_sql(sql7b, conn)

,first,last,addr,zip,cust_id,prod_id,prod_desc
0,Fode/Beed,Vincent,8509 Route 44,97846,1,501,Washer
1,Fode/Beed,Vincent,8509 Route 44,97846,1,502,Nail
2,Fode/Beed,Vincent,8509 Route 44,97846,1,503,Nut
3,Fode/Beed,Vincent,8509 Route 44,97846,1,504,Screw
4,Fode/Beed,Vincent,8509 Route 44,97846,1,505,Bolt
...,...,...,...,...,...,...,...
6911,Padme,Le,1031 Pheasant Run,89178,250,524,Axe
6912,C-3Po,Miller,9992 Heather Lane,65768,251,509,Clamp
6913,C-3Po,Miller,9992 Heather Lane,65768,251,522,Hacksaw
6914,C-3Po,Miller,9992 Heather Lane,65768,251,523,Saw


In [70]:
# every customer with every product_id
sql7c = """
SELECT first,
        last,
        address as addr,
        zip, 
        cust_id,
        prod_id,
        prod_desc
FROM tCust
CROSS JOIN tProd
;"""
pd.read_sql(sql7c, conn)

,first,last,addr,zip,cust_id,prod_id,prod_desc
0,Fode/Beed,Vincent,8509 Route 44,97846,1,501,Washer
1,Fode/Beed,Vincent,8509 Route 44,97846,1,502,Nail
2,Fode/Beed,Vincent,8509 Route 44,97846,1,503,Nut
3,Fode/Beed,Vincent,8509 Route 44,97846,1,504,Screw
4,Fode/Beed,Vincent,8509 Route 44,97846,1,505,Bolt
...,...,...,...,...,...,...,...
7525,C-3Po,Miller,9992 Heather Lane,65768,251,526,Wire
7526,C-3Po,Miller,9992 Heather Lane,65768,251,527,Toolbox
7527,C-3Po,Miller,9992 Heather Lane,65768,251,528,Ladder
7528,C-3Po,Miller,9992 Heather Lane,65768,251,529,Workbench


In [146]:
sql7 = """
WITH CustProdPurchase AS
(SELECT first,
        last,
        address as addr,
        zip, 
        cust_id,
        prod_id,
        prod_desc
FROM tCust
JOIN tInvoice USING (cust_id)
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY cust_id, prod_id
),

AllCustProdCombo AS
(SELECT first,
        last,
        address as addr,
        zip, 
        cust_id,
        prod_id,
        prod_desc
FROM tCust
CROSS JOIN tProd
)

SELECT first,
        last,
        addr,
        zip, 
        prod_id,
        prod_desc
FROM AllCustProdCombo
WHERE (cust_id, prod_id) NOT IN (SELECT cust_id, prod_id FROM CustProdPurchase)
ORDER BY first ASC, last ASC, addr ASC, zip ASC, prod_id ASC
;"""
pd.read_sql(sql7, conn)

,first,last,addr,zip,prod_id,prod_desc
0,Bala-Tik,Benson,3204 Jackson Avenue,99778,529,Workbench
1,Bala-Tik,Benson,3204 Jackson Avenue,99778,530,Chainsaw
2,Bala-Tik,Elliott,9088 Valley Road,55003,504,Screw
3,Bala-Tik,Elliott,9088 Valley Road,55003,513,Hammer
4,Bala-Tik,Elliott,9088 Valley Road,55003,515,Wrench
...,...,...,...,...,...,...
609,Wedge,Gordon,2566 2nd Street,33196,527,Toolbox
610,Wedge,Gordon,2566 2nd Street,33196,529,Workbench
611,Wedge,Gordon,2566 2nd Street,33196,530,Chainsaw
612,Wedge,Schmidt,8114 Valley Road,89427,530,Chainsaw


In [147]:
my_sql['q7']['sql'] = sql7

---

8) For each customer, report the total sales from their most recent invoice.

Return:

- first
- last
- addr
- zip
- invoice_id (their most recent invoice)
- total sales for their most recent invoice

Order the results by first, last, addr, zip - all ascending

In [84]:
sql8a = """
SELECT cust_id,
        MAX(invoice_id) as invoice_id
FROM tInvoice
GROUP BY cust_id
;"""
pd.read_sql(sql8a, conn)

,cust_id,invoice_id
0,1,28305
1,2,28420
2,3,28523
3,4,27320
4,5,28185
...,...,...
246,247,28485
247,248,28223
248,249,28506
249,250,28493


In [150]:
sql8 = """
WITH MostRecentInvoice AS
(SELECT cust_id,
        MAX(invoice_id) as invoice_id
FROM tInvoice
GROUP BY cust_id
)

SELECT first,
        last,
        address as addr,
        zip,
        invoice_id,
        SUM(qty * unit_price)*1.0/100 AS "total sales"
FROM tCust
JOIN tInvoice USING (cust_id)
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
WHERE (cust_id, invoice_id) IN (SELECT cust_id, invoice_id FROM MostRecentInvoice)
GROUP BY invoice_id
ORDER BY first ASC, last ASC, addr ASC, zip ASC
;"""
pd.read_sql(sql8, conn)

,first,last,addr,zip,invoice_id,total sales
0,Bala-Tik,Benson,3204 Jackson Avenue,99778,28207,105.81
1,Bala-Tik,Elliott,9088 Valley Road,55003,26207,90.09
2,Bala-Tik,Ingram,1819 Pheasant Run,676,18967,103.22
3,Bala-Tik,Lara,4007 Valley Road,27405,28422,3.00
4,Bala-Tik,Miller,8714 Washington Street,30286,28464,110.64
...,...,...,...,...,...,...
246,Unkar Plutt,Walker,2416 Pheasant Run,68030,28269,158.24
247,Wedge,Elliott,7110 Strawberry Lane,3830,28301,69.00
248,Wedge,Gordon,2566 2nd Street,33196,18405,94.34
249,Wedge,Schmidt,8114 Valley Road,89427,28259,13.00


In [152]:
my_sql['q8']['sql'] = sql8

---

9) The marketing department keeps calling with all these annoying ideas...

Now they want you to find the state which had the lowest sales last month and return a list of all products we didn't sell in that state last month!

As usual, don't hardcode in any dates or states. Make sure this query can be run every month moving forward without any adjustments to the code.

Return:

- prod_id
- prod_desc

Order the results by prod_id ASC

In [99]:
sql9a = """
SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
;"""
pd.read_sql(sql9a, conn)

,year,month
0,2024,11


In [109]:
# now apply this to last month
sql9b = """
SELECT state_id,
        SUM(qty * unit_price)*1.0/100 AS "total sales"
FROM tZip
JOIN tCust USING (zip)
JOIN tInvoice USING (cust_id)
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY state_id
ORDER BY "total sales" ASC
LIMIT 1
;"""
pd.read_sql(sql9b, conn)

,state_id,total sales
0,AL,13749.32


In [123]:
# products sold in lowest sales state
sql9c = """
WITH LastMonth AS
(SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
),

LowestSalesState AS
(SELECT state_id,
        SUM(qty * unit_price)*1.0/100 AS "total sales"
FROM tZip
JOIN tCust USING (zip)
JOIN tInvoice USING (cust_id)
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
WHERE (year, month) IN (SELECT year, month FROM LastMonth)
GROUP BY state_id
ORDER BY "total sales" ASC
LIMIT 1
)

SELECT DISTINCT prod_id
FROM tProd
JOIN tInvoiceDetail USING (prod_id)
JOIN tInvoice USING (invoice_id)
JOIN tCust USING (cust_id)
JOIN tZip USING (zip)
WHERE state_id = (SELECT state_id FROM LowestSalesState)
    AND (year, month) IN (SELECT year, month FROM LastMonth)
ORDER BY prod_id ASC
;"""
pd.read_sql(sql9c, conn)

,prod_id
0,505
1,506
2,507
3,510
4,511
5,513
6,514
7,516
8,517
9,520


In [153]:
sql9 = """
WITH LastMonth AS
(SELECT DISTINCT year, month
FROM tInvoice
ORDER BY year DESC, month DESC
LIMIT 1
),

LowestSalesState AS
(SELECT state_id,
        SUM(qty * unit_price)*1.0/100 AS "total sales"
FROM tZip
JOIN tCust USING (zip)
JOIN tInvoice USING (cust_id)
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
WHERE (year, month) IN (SELECT year, month FROM LastMonth)
GROUP BY state_id
ORDER BY "total sales" ASC
LIMIT 1
),

ProdSoldInState AS
(
SELECT DISTINCT prod_id
FROM tProd
JOIN tInvoiceDetail USING (prod_id)
JOIN tInvoice USING (invoice_id)
JOIN tCust USING (cust_id)
JOIN tZip USING (zip)
WHERE state_id = (SELECT state_id FROM LowestSalesState)
    AND (year, month) IN (SELECT year, month FROM LastMonth)
),

AllProd AS
(SELECT prod_id,
        prod_desc
FROM tProd
)

SELECT prod_id,
        prod_desc
FROM AllProd
WHERE prod_id NOT IN (SELECT prod_id FROM ProdSoldInState)
ORDER BY prod_id ASC
;"""
pd.read_sql(sql9, conn)

,prod_id,prod_desc
0,501,Washer
1,502,Nail
2,503,Nut
3,504,Screw
4,508,Bradawl
5,509,Clamp
6,512,Scraper
7,515,Wrench
8,518,Level
9,519,Monkeywrench


In [154]:
my_sql['q9']['sql'] = sql9

In [155]:
conn.close()

In [156]:
# View questions for submission
def print_dict_items(my_dict, prepend_key = None):
    # Convert to standard dict type
    d = dict(my_dict)

    # Loop over the key, value pairs
    for key, value in d.items():
        # If a key to prepend was passed,
        # put it in front of the current key
        if prepend_key is not None:
            key = f'{prepend_key} : {key}'

        # If the value is another dictionary,
        # call the function recursively and pass the current key to prepend
        if type(value) == dict:
            print_dict_items(value, key)
        # Otherwise, print the key and value
        else:
            if type(value) == list:
                print(f'Key: {key}\nValues:')
                for val in value:
                    print(f'{val}\n')
                print('----------')
            else:
                print(f'Key: {key}\nValue: {value}\n----------')
    return

print_dict_items(my_sql)

Key: setup
Values:
----------
Key: q1 : sql
Value: 
WITH InvoiceData AS
(SELECT year,
        month,
        qty,
        unit_price,
        invoice_id,
        cust_id
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
)

SELECT year,
        month,
        SUM(qty * unit_price)/100 AS Sales,
        COUNT(DISTINCT invoice_id) AS NumInvoices,
        COUNT(DISTINCT cust_id) AS NumCust,
        COUNT(DISTINCT invoice_id)*1.0/COUNT(DISTINCT cust_id) AS InvoicesPerCust,
        (SUM(qty * unit_price)/100)*1.0/COUNT(DISTINCT cust_id) AS SalesPerCust,
        (SUM(qty * unit_price)/100)*1.0/COUNT(DISTINCT invoice_id) AS SalesPerInvoice
FROM InvoiceData
GROUP BY year, month
ORDER BY year ASC, month ASC
;
----------
Key: q2 : sql
Value: 
WITH InvoiceData AS
(SELECT year,
        month,
        (SUM(qty * unit_price)/100) AS sales
FROM tInvoice
JOIN tInvoiceDetail USING (invoice_id)
JOIN tProd USING (prod_id)
GROUP BY year, month
),

MinSalesPerYear AS
(SELECT ye

In [157]:
# The name for the output file. It MUST end in .data302
submission_name = 'Final.data302'

# This will export the contents of the my_sql dictionary to the file named above
with open(submission_name, 'w') as f:
    json.dump(my_sql, f)

In [158]:
# Validate the submission format, and then inspect the report it generates

# Path to the database:
path_db = 'data/hardware.sqlite'

# Run the validation tool
# Make sure to set num_questions to the correct value for the assignment.
validate_submission_format(path_db, 
                           submission_name, 
                           num_questions=9
                          )

Report saved to:  VALIDATION_Final.txt
